In [1]:
from gec_common.gecclass import *
from gec_common import funcation as fn
from gec_common import json_folder as js
# import gec_common.OutputJSON
SCRIPT_NAME = "movie_data"
output_json_file = js.OutputJSON(SCRIPT_NAME)
output_json_folder = "json_file"
import re
import jsons
import time
import os
from datetime import date, datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from deep_translator import GoogleTranslator
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
notice_count = 0
def extract_and_save_notice(tender_html_element):
    global notice_count
    global notice_data
    notice_data = tender()
    try:
        notice_data.sub_url = tender_html_element.get_attribute('href')
        print('sub_url::',notice_data.sub_url)
    except Exception as e:
        print("Exception in sub_url: {}".format(type(e).__name__))
        pass
#     sub_page.get(notice_data.sub_url)
    fn.load_page(sub_page, notice_data.sub_url, 60)
    time.sleep(10)
    try:
        notice_data.movie_title = sub_page.find_element(By.CSS_SELECTOR, ' div.title.ott_false > h2 > a').text
        print(notice_data.movie_title)
#         movie_title = GoogleTranslator(source='auto', target='en').translate(local_title)
    except Exception as e:
        print("Exception in movie_title: {}".format(type(e).__name__))
        pass
    try:
        notice_data.country_code = sub_page.find_element(By.CSS_SELECTOR, "span.release").text.split('(')[1].split(')')[0].strip()
        print('country_code:;',notice_data.country_code)
    except Exception as e:
        print("Exception in country_code: {}".format(type(e).__name__))
        pass
    try:
        release_date = sub_page.find_element(By.CSS_SELECTOR, "span.release").text
        release_date = re.findall('\d{4}-\d+-\d+',release_date)[0]
        notice_data.release_date = datetime.strptime(release_date,'%Y-%m-%d')#.strftime('%Y/%m/%d %H:%M:%S')
        print(notice_data.release_date)
    except Exception as e:
        print("Exception in release_date: {}".format(type(e).__name__))
        pass
#     if notice_data.publish_date is not None and notice_data.publish_date < threshold:
#         return
    try:
        duration = sub_page.find_element(By.CSS_SELECTOR, "span.runtime").text
        notice_data.duration = re.findall('\d+h \d+m',duration)[0]
        print(notice_data.duration)
    except Exception as e:
        print("Exception in duration: {}".format(type(e).__name__))
        pass
    try:
        user_score = sub_page.find_element(By.XPATH, "//*[@class='user_score_chart']").get_attribute('data-percent')
        notice_data.user_score =float(user_score.replace(',','').strip())
        print("user_score::",notice_data.user_score)
    except Exception as e:
        print("Exception in user_score: {}".format(type(e).__name__))
        pass  
    try:
        notice_data.types = sub_page.find_element(By.XPATH, "(//*[@class='genres'])[1]").text
        print("types::",notice_data.types)
    except Exception as e:
        print("Exception in types: {}".format(type(e).__name__))
        pass
    try:
        directore = sub_page.find_element(By.XPATH, "(//*[@class='profile'])[1]").text
        if "Director" in directore:
            notice_data.directore_name =directore
        elif "Writer" in directore:
            notice_data.writer_name =directore
        print("writer_name::",notice_data.writer_name)
        print("directore_name::",notice_data.directore_name)
    except Exception as e:
        print("Exception in directore_name: {}".format(type(e).__name__))
        pass
    try:
        writer = sub_page.find_element(By.XPATH, "(//*[@class='profile'])[2]").text
        if "Director" in writer:
            notice_data.directore_name =writer
        elif "Writer" in writer:
            notice_data.writer_name =writer
        print("writer_name::",notice_data.writer_name)
        print("directore_name::",notice_data.directore_name)
    except Exception as e:
        print("Exception in writer_name: {}".format(type(e).__name__))
        pass
    try:
        notice_data.overview = sub_page.find_element(By.XPATH, "(//*[@class='overview'])[1]").text
        print("overview::",notice_data.overview)
    except Exception as e:
        print("Exception in overview: {}".format(type(e).__name__))
        pass
    try:
        notice_data.reviews = int(sub_page.find_element(By.CSS_SELECTOR, "#reviews > span").text)
        print('review::',notice_data.reviews)
    except Exception as e:
        print("Exception in reviews: {}".format(type(e).__name__))
        pass
    try:
        notice_data.status = sub_page.find_element(By.CSS_SELECTOR, "section.facts.left_column > p:nth-child(2)").text.split('Status')[1].strip()
        print("status::",notice_data.status)
    except Exception as e:
        print("Exception in status: {}".format(type(e).__name__))
        pass
    try:
        notice_data.original_language = sub_page.find_element(By.CSS_SELECTOR, "section.facts.left_column > p:nth-child(3)").text.split('Original Language')[1].strip()
        print("original_language::",notice_data.original_language)
    except Exception as e:
        print("Exception in original_language: {}".format(type(e).__name__))
        pass
    try:
        budgets = sub_page.find_element(By.CSS_SELECTOR, "section.facts.left_column > p:nth-child(4)").text.split('Budget')[1].strip()
        print("budgets::",budgets)
        budgets = re.sub("[^\d\.\,]","",budgets)
        notice_data.budget = float(budgets.replace(',','').strip())
    except Exception as e:
        print("Exception in budget: {}".format(type(e).__name__))
        pass
    try:
        revenues = sub_page.find_element(By.CSS_SELECTOR, "section.facts.left_column > p:nth-child(5)").text.split('Revenue')[1].strip()
        print("revenues::",revenues)
        revenues = re.sub("[^\d\.\,]","",revenues)
        notice_data.revenue = float(revenues.replace(',','').strip())
        print("Revenue::",notice_data.revenue)
    except Exception as e:
        print("Exception in revenues: {}".format(type(e).__name__))
        pass

    notice_data.tender_cleanup()
    notice_count+=1
    output_json_file.writeNoticeToJSONFile(jsons.dump(notice_data))

# ----------------------------------------- Main Body
arguments= ['--incognito','ignore-certificate-errors','allow-insecure-localhost','--start-maximized']
page_main =fn.init_chrome_driver_head(arguments)
sub_page =fn.init_chrome_driver_head(arguments)
try:
    th = date.today()
    threshold = th.strftime('%Y/%m/%d')
    print("Scraping from or greater than: " + threshold)
    urls = ["https://www.themoviedb.org/movie?page="] 
    for url in urls:
        for no in range(1,2):
            urls =url+str(no)
            fn.load_page(page_main, urls, 60)
            time.sleep(5)
            print('----------------------------------')
            print(urls)
            time.sleep(5)

            rows = WebDriverWait(page_main, 60).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div > div.content > h2 > a')))
            length = len(rows)
            for records in range(0,1):
                tender_html_element = WebDriverWait(page_main, 60).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div > div.content > h2 > a')))[records]
                extract_and_save_notice(tender_html_element)
    print("Finished processing. Scraped {} notices".format(notice_count))
except Exception as e:
    print("Exception:"+str(e))
    pass
#     page_main.quit()

output_json_file.copyFinalJSONToServer(output_json_folder)

Scraping from or greater than: 2024/05/19
----------------------------------
https://www.themoviedb.org/movie?page=1
sub_url:: https://www.themoviedb.org/movie/823464-godzilla-x-kong-the-new-empire
Exception in movie_title: NoSuchElementException
country_code:; IN
2024-03-29 00:00:00
1h 55m
user_score:: 72.0
types:: Science Fiction, Action, Adventure
writer_name:: None
directore_name:: Adam Wingard
Director, Story
writer_name:: None
directore_name:: Adam Wingard
Director, Story
overview:: Following their explosive showdown, Godzilla and Kong must reunite against a colossal undiscovered threat hidden within our world, challenging their very existence – and our own.
review:: 7
status:: Released
original_language:: English
budgets:: $150,000,000.00
revenues:: $558,503,759.00
Revenue:: 558503759.0
Finished processing. Scraped 1 notices


C:\Users\addiction computers\anaconda3\Lib\site-packages\jsons\_common_impl.py:43: UserWarning: The use of datetimes without timezone is dangerous and can lead to undesired results. Use suppress_warning(datetime-without-tz) or suppress_warnings(True) to turn off this message.
  warnings.warn(msg_, *args, **kwargs)


In [ ]:
s= dict{}